In [16]:
import requests
import string
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from bs4.element import ResultSet
import os

# Functions

In [2]:
def track_tropes_crawled(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    td_tags = soup.find_all('td')
    trope_names = []
    trope_links = []
    for td in td_tags:
        a_tag = td.find('a', href=True)
        if a_tag:
            link = a_tag['href']
            trope_links.append(link)
            link_text = a_tag.get_text(strip=True)
            trope_names.append(link_text)
    return pd.DataFrame({'trope_name':trope_names, 'trope_link':trope_links})

In [3]:
def parsing_trope_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the main content div with the id 'main-content'
    main_content = soup.find('div', id='main-content')
    if not main_content:
        print("No 'main-content' found.")
        return None, None, None
    # Find the article with id 'main-entry'
    main_entry = main_content.find('article', id='main-entry')

    if not main_entry:
        print("No 'main-entry' found.")
        return None, None, None

    # Initialize a list to store paragraphs
    paragraphs = []
    related_links = []
    related_link_names = []
    example_descriptions = []
    example_related_links = []
    example_related_link_names = []
    found_examples = False

    # Loop through all elements in the main entry
    for elem in main_entry.find_all(True):
        if elem.name == 'h2':
          break
        if elem.name == 'p':
          paragraph_text = ''
          for content in elem.contents:
              if content.name == 'a' and 'href' in content.attrs:
                  link = content['href']
                  link_text = content.get_text(strip=True)
                  paragraph_text += f' {link_text} '
                  related_links.append(link)
                  related_link_names.append(link_text)
              else:
                    paragraph_text += content.strip() if isinstance(content, str) else ''
          paragraphs.append(paragraph_text.strip())

    for elem in main_entry.find_all(True):
        if elem.name == 'h2':
            if 'Examples' in elem.get_text() or 'Other examples' in elem.get_text():
                found_examples = True
                continue
        if found_examples:
          if 'folder' in elem.get('class',[]):
            list_items = elem.find_all('li')
            for li in list_items:
              example_text = ''
              for content in li.contents:
                if content.name == 'a' and 'href' in content.attrs:
                  link = content['href']
                  link_text = content.get_text(strip=True)
                  example_text += f' {link_text} '
                  example_related_links.append(link)
                  example_related_link_names.append(link_text)
                else:
                  example_text += content.strip() if isinstance(content, str) else ''
                if content.name == 'em':
                  for em_content in content.contents:
                    if em_content.name == 'a' and 'href' in em_content.attrs:
                      link = em_content['href']
                      link_text = em_content.get_text(strip=True)
                      example_text += f' {link_text} '
                      example_related_links.append(link)
                      example_related_link_names.append(link_text)
                    else:
                      example_text += em_content.strip() if isinstance(em_content, str) else ''
              example_descriptions.append(example_text.strip())
    # Print the paragraphs
    if not paragraphs:
        print("No paragraphs found before the 'Examples' section.")

    if not example_descriptions:
        print("No example descriptions found.")
        print(len(example_descriptions))

    return paragraphs, related_links, related_link_names, example_descriptions, example_related_links, example_related_link_names


# Widespread Trope Collecting

In [4]:
def parsing_with_csv_aid(df, output_folder):

  if os.path.exists(output_folder):
    output_df = pd.read_csv(output_folder)
    print("Created CSV!")
  else:
    output_df = pd.DataFrame(columns=df.columns)
    print("Found CSV!")
  for index, row in df.iterrows():
    if index in output_df.index:
      print(f"Trope '{row['trope_name']}' already exists, skipping.")
      continue
    try:
      name = row['trope_name']
      url = row['trope_link']
      #example_descriptions, example_related_links, example_related_link_names
      row['trope_description'], row['related_link_names'], row['related_links'],\
       row['example_descriptions'], row['example_related_links'], row['example_related_link_names']= parsing_trope_page(url)

      output_df = pd.concat([output_df, pd.DataFrame([row])])

      output_df.to_csv(output_folder, index=False)
      print(f"Successfully processed and saved: {name}")

    except requests.exceptions.RequestException as e:
          print(f"Failed to process {name} at row {index}: {e}")
          break  # Stop the loop if there is an error
    # Ensuring the buffer is flushed
    with open(output_folder, 'a') as f:
        f.flush()
  return output_df

In [5]:
def parsing_with_csv_aid(df, output_folder):

  if os.path.exists(output_folder):
    output_df = pd.read_csv(output_folder)
    print("Created CSV!")
  else:
    output_df = pd.DataFrame(columns=df.columns)
    print("Found CSV!")
  previous_letter = None
    
  for index, row in df.iterrows():
    if index in output_df.index:
      print(f"Already Processed {row['trope_name']}")
      continue
    try:
      name = row['trope_name']
      url = row['trope_link']
      #example_descriptions, example_related_links, example_related_link_names
      row['trope_description'], row['related_link_names'], row['related_links'],\
       row['example_descriptions'], row['example_related_links'], row['example_related_link_names']= parsing_trope_page(url)
        
      output_df = pd.concat([output_df, pd.DataFrame([row])])
      current_letter = name[0].upper()
      if current_letter != previous_letter:
        print("Making Iterative save")
        filename = f'../data_scraping_storage/Up to {current_letter}.csv'
        output_df.to_csv(filename, index=False)
        previous_letter = current_letter
      
      output_df.to_csv(output_folder, index=False)
      print(f"Successfully processed and saved: {name}")

    except requests.exceptions.RequestException as e:
          print(f"Failed to process {name} at row {index}: {e}")
          break  # Stop the loop if there is an error
    except Exception as e:
        print(f'Caught unexpected error {e}')
        break
  return output_df

In [6]:
output_folder = '../data_raw/parsed_tropes.csv'


In [7]:
df = pd.read_csv('../data_raw/all_trope_names_links.csv')

In [8]:
final_df = parsing_with_csv_aid(df, output_folder)

Created CSV!
Already Processed Aardvark Trunks
Already Processed Abandoned Area
Already Processed Abandoned Camp Ruins
Already Processed Abandoned Catchphrase
Already Processed Abandoned Hospital
Already Processed Abandoned Hospital Awakening
Already Processed Abandoned Info Page
Already Processed Abandoned Laboratory
Already Processed Abandoned Mascot
Already Processed Abandoned Mine
Already Processed Abandoned Pet In A Box
Already Processed Abandoned Playground
Already Processed Abandoned War Child
Already Processed Abandoned Warehouse
Already Processed Abandonment Induced Animosity
Already Processed Abandon Ship
Already Processed Abandon Shipping
Already Processed Abandon The Disabled
Already Processed Abandonware
Already Processed Abbey Road Crossing
Already Processed Abdicate The Throne
Already Processed Abduction Is Love
Already Processed A Beast In Name And Nature
Already Processed Abhorrent Admirer
Already Processed Abilene Paradox
Already Processed Ability Depletion Penalty
Al

Already Processed Happy Rain
Already Processed Harassing Phone Call
Already Processed Harbinger Of Impending Doom
Already Processed Hardboiled Detective
Already Processed Hard Coded Hostility
Already Processed Hard Core
Already Processed Hardcore Hip Hop
Already Processed Hardcore Punk
Already Processed Hardcore Techno
Already Processed Hard Cut
Already Processed Hard Drinking Party Girl
Already Processed Harder Than Hard
Already Processed Hard Head
Already Processed Hard Levels Easy Bosses
Already Processed Hard Light
Already Processed Hard Mode Filler
Already Processed Hard Mode Mook
Already Processed Hard Mode Perks
Already Processed Hard On Soft Science
Already Processed Hard Rock
Already Processed Hard To Adapt Work
Already Processed Hard To Light Fire
Already Processed Hard Truckin
Already Processed Hard Truth Aesop
Already Processed Hard Work Fallacy
Already Processed Hard Work Hardly Works
Already Processed Hard Work Montage
Already Processed Harem Genre
Already Processed Harem

Already Processed The Cavalry
Already Processed The Cavalry Arrives Late
Already Processed The Centerpiece Spectacular
Already Processed The Chain Of Command
Already Processed The Chain Of Harm
Already Processed The Chains Of Commanding
Already Processed The Champion
Already Processed The Chanteuse
Already Processed The Character Died With Him
Already Processed The Character Ice Cream Bar
Already Processed The Charmer
Already Processed The Chase
Already Processed The Chessmaster
Already Processed The Chew Toy
Already Processed The Chiefs Daughter
Already Processed The Chikan
Already Processed The Chooser Of The One
Already Processed The Chosen Many
Already Processed The Chosen One
Already Processed The Chosen People
Already Processed The Chosen Wannabe
Already Processed The Chosen Zero
Already Processed The Chris Carter Effect
Already Processed The Christmas Annual
Already Processed The Church
Already Processed The City Narrows
Already Processed The City Vs The Country
Already Processe

Making Iterative save
Successfully processed and saved: Vacation Crossover
Successfully processed and saved: Vacation Dear Boy
Successfully processed and saved: Vacation Episode
Successfully processed and saved: Vacuum Mouth
Successfully processed and saved: Vader Breath
No example descriptions found.
0
Successfully processed and saved: Vagabond Buddies
Successfully processed and saved: Vagina Dentata
Successfully processed and saved: Vague Age
Successfully processed and saved: Vague Hit Points
Successfully processed and saved: Vagueness Is Coming
Successfully processed and saved: Vague Stat Values
Successfully processed and saved: Vain Sorceress
Successfully processed and saved: Valentines Day Episode
Successfully processed and saved: Valentines Day Violence
Successfully processed and saved: Valentines Day Vitriol
Successfully processed and saved: Valkyries
Successfully processed and saved: Valley Girl
No example descriptions found.
0
Successfully processed and saved: Values Dissonanc

Successfully processed and saved: Victorian Novel Disease
Successfully processed and saved: Victorias Secret Compartment
Successfully processed and saved: Victorious Chorus
Successfully processed and saved: Victorious Loser
Successfully processed and saved: Victorious Roar
Successfully processed and saved: Victor Steals Insignia
Successfully processed and saved: Victory By Endurance
Successfully processed and saved: Victory By First Blood
Successfully processed and saved: Victory Fakeout
Successfully processed and saved: Victory Guided Amnesia
Successfully processed and saved: Victory Is Boring
Successfully processed and saved: Victory Pose
Successfully processed and saved: Victory Pose On Person
Successfully processed and saved: Victory Quote
Successfully processed and saved: Victory Sex
Successfully processed and saved: Victory Through Intimidation
Successfully processed and saved: Video Arcade
Successfully processed and saved: Video Call Fail
Successfully processed and saved: Video 

...

Successfully processed and saved: Whos Laughing Now
Successfully processed and saved: Whos On First
No example descriptions found.
0
Successfully processed and saved: Whos Watching The Store
Successfully processed and saved: Whos Your Daddy
No example descriptions found.
0
Successfully processed and saved: Who Wants To Be Who Wants To Be A Millionaire
Successfully processed and saved: Who Wants To Live Forever
Successfully processed and saved: Who Watches The Watchmen
Successfully processed and saved: Who Will Bell The Cat
Successfully processed and saved: Who Will Take The Kids
Successfully processed and saved: Who Would Be Stupid Enough
Successfully processed and saved: Who Would Want To Watch Us
Successfully processed and saved: Who Writes This Crap
Successfully processed and saved: Who You Gonna Call
Successfully processed and saved: Why Am I Ticking
Successfully processed and saved: Why Are We Whispering
Successfully processed and saved: Why Are You Looking At Me Like That
Success

Successfully processed and saved: With Great Power Comes Great Insanity
Successfully processed and saved: With Great Power Comes Great Opposition
Successfully processed and saved: With Great Power Comes Great Perks
Successfully processed and saved: With Great Power Comes Great Responsibility
Successfully processed and saved: Withholding The Big Good
Successfully processed and saved: Withholding The Cure
Successfully processed and saved: Withholding Their Name
Successfully processed and saved: Within Arms Reach
Successfully processed and saved: Within Parameters
Successfully processed and saved: With Lyrics
Successfully processed and saved: With My Dying Breath I Summon You
Successfully processed and saved: With My Hands Tied
Successfully processed and saved: With Or Without You
Successfully processed and saved: With Our Swords Scene
Successfully processed and saved: With This Herring
Successfully processed and saved: With This Ring
Successfully processed and saved: With Us Or Against U

Successfully processed and saved: Worse With Context
Successfully processed and saved: Worshipped For Great Deeds
Successfully processed and saved: Worst Aid
Successfully processed and saved: Worst News Judgment Ever
Successfully processed and saved: Worst Wedding Ever
Successfully processed and saved: Worst Whatever Ever
Successfully processed and saved: Worth It
Successfully processed and saved: Worthless Currency
Successfully processed and saved: Worthless Foreign Degree
Successfully processed and saved: Worthless Treasure Twist
Successfully processed and saved: Worthless Yellow Rocks
Successfully processed and saved: Worth Living For
Successfully processed and saved: Worthy Opponent
Successfully processed and saved: Would Be Rude To Say Genocide
Successfully processed and saved: Would Harm A Senior
Successfully processed and saved: Would Hit A Girl
Successfully processed and saved: Would Hurt A Child
Successfully processed and saved: Would Not Shoot A Civilian
Successfully processe

Successfully processed and saved: Yellow Peril
Successfully processed and saved: Yellow Purple Contrast
Successfully processed and saved: Yellow Sash Of Power
Successfully processed and saved: Yellow Snow
No example descriptions found.
0
Successfully processed and saved: Ye Olde Butcherede Englishe
Successfully processed and saved: Ye Olde Nuclear Silo
Successfully processed and saved: Yes Man
Successfully processed and saved: Yes No Answer Interpretation
Successfully processed and saved: Yet Another Baby Panda
No example descriptions found.
0
Successfully processed and saved: Yet Another Christmas Carol
No example descriptions found.
0
Successfully processed and saved: Yet Another Stupid Death
No example descriptions found.
0
Successfully processed and saved: Yiddish As A Second Language
No example descriptions found.
0
Successfully processed and saved: Yiff
Successfully processed and saved: Yin Yang Bomb
Successfully processed and saved: Yin Yang Clash
Successfully processed and save

No example descriptions found.
0
Successfully processed and saved: Young Future Famous People
Successfully processed and saved: Young Gun
Successfully processed and saved: Young Love Versus Old Hate
Successfully processed and saved: You No Take Candle
No example descriptions found.
0
Successfully processed and saved: You Nuke Em
Successfully processed and saved: You Owe Me
Successfully processed and saved: You Put The X In XY
Successfully processed and saved: Your Answer To Everything
Successfully processed and saved: Your Approval Fills Me With Shame
Successfully processed and saved: Your Brain Wont Be Much Of A Meal
Successfully processed and saved: Your Costume Needs Work
Successfully processed and saved: Your Days Are Numbered
Successfully processed and saved: Your Door Was Open
Successfully processed and saved: You Rebel Scum
Successfully processed and saved: Youre Cute When Youre Angry
Successfully processed and saved: Youre Drinking Breast Milk
Successfully processed and saved: 

In [9]:
final_df.tail()

,trope_name,trope_link,trope_description,related_link_names,related_links,example_descriptions,example_related_links,example_related_link_names
29898,Zombify The Living,http://tvtropes.org/pmwiki/pmwiki.php/Main/Zom...,[http://tvtropes.org/pmwiki/pmwiki.php/Main/Zo...,"[/pmwiki/pmwiki.php/Main/AnimateDead, /pmwiki/...","[Animating dead bodies, summoning skeletons, S...",[Sankarea touches on this trope. Although Chih...,"[/pmwiki/pmwiki.php/Manga/Sankarea, /pmwiki/pm...","[Sankarea, The Power of Love, Jo Jos Bizarre A..."
29899,Zonk,http://tvtropes.org/pmwiki/pmwiki.php/Main/Zonk,[http://tvtropes.org/pmwiki/pmwiki.php/Main/Zo...,"[/pmwiki/pmwiki.php/Main/GameShow, /pmwiki/pmw...","[Game Shows, Rule of Funny, something valuable...",[You can buy most of those food-type Zonks at ...,"[/pmwiki/pmwiki.php/Main/WorthlessYellowRocks,...","[didn't know this, a red T-shirt emblazoned wi..."
29900,Zoom,http://tvtropes.org/pmwiki/pmwiki.php/Main/Zoom,[http://tvtropes.org/pmwiki/pmwiki.php/Main/Zo...,"[/pmwiki/pmwiki.php/MediaNotes/FocalLength, /p...","[Focal Length, dolly, Lens Flare, Establishing...",[],[],[]
29901,Zoomed Out Reframing Gag,http://tvtropes.org/pmwiki/pmwiki.php/Main/Zoo...,[http://tvtropes.org/pmwiki/pmwiki.php/Main/Zo...,"[/pmwiki/pmwiki.php/Main/Comedy, /pmwiki/pmwik...","[comedy, absurdity of a situation is highlight...",[],[],[]
29902,Zorro Mark,http://tvtropes.org/pmwiki/pmwiki.php/Main/Zor...,[http://tvtropes.org/pmwiki/pmwiki.php/Main/Zo...,"[/pmwiki/pmwiki.php/Main/TropeNamers, /pmwiki/...","[Trope Namer, Zorro, Calling Card, Sub-Trope, ...",[Corny Snaps was a Kellogg's breakfast cereal ...,"[/pmwiki/pmwiki.php/Literature/Zorro, /pmwiki/...","[Zorro-like, Anpanman, Zorro, Bubblegum Crisis..."


In [10]:
final_df.to_csv('just_in_case_csv.csv',index=False)

In [11]:
df = pd.read_csv(output_folder)

In [12]:
df.tail()

,trope_name,trope_link,trope_description,related_link_names,related_links,example_descriptions,example_related_links,example_related_link_names
29898,Zombify The Living,http://tvtropes.org/pmwiki/pmwiki.php/Main/Zom...,['http://tvtropes.org/pmwiki/pmwiki.php/Main/Z...,"['/pmwiki/pmwiki.php/Main/AnimateDead', '/pmwi...","['Animating dead bodies', 'summoning skeletons...",['Sankarea touches on this trope. Although Chi...,"['/pmwiki/pmwiki.php/Manga/Sankarea', '/pmwiki...","['Sankarea', 'The Power of Love', 'Jo Jos Biza..."
29899,Zonk,http://tvtropes.org/pmwiki/pmwiki.php/Main/Zonk,['http://tvtropes.org/pmwiki/pmwiki.php/Main/Z...,"['/pmwiki/pmwiki.php/Main/GameShow', '/pmwiki/...","['Game Shows', 'Rule of Funny', 'something val...","[""You can buy most of those food-type Zonks at...",['/pmwiki/pmwiki.php/Main/WorthlessYellowRocks...,"[""didn't know this"", 'a red T-shirt emblazoned..."
29900,Zoom,http://tvtropes.org/pmwiki/pmwiki.php/Main/Zoom,['http://tvtropes.org/pmwiki/pmwiki.php/Main/Z...,"['/pmwiki/pmwiki.php/MediaNotes/FocalLength', ...","['Focal Length', 'dolly', 'Lens Flare', 'Estab...",[],[],[]
29901,Zoomed Out Reframing Gag,http://tvtropes.org/pmwiki/pmwiki.php/Main/Zoo...,['http://tvtropes.org/pmwiki/pmwiki.php/Main/Z...,"['/pmwiki/pmwiki.php/Main/Comedy', '/pmwiki/pm...","['comedy', 'absurdity of a situation is highli...",[],[],[]
29902,Zorro Mark,http://tvtropes.org/pmwiki/pmwiki.php/Main/Zor...,['http://tvtropes.org/pmwiki/pmwiki.php/Main/Z...,"['/pmwiki/pmwiki.php/Main/TropeNamers', '/pmwi...","['Trope Namer', 'Zorro', 'Calling Card', 'Sub-...",['Corny Snaps was a Kellogg\'s breakfast cerea...,"['/pmwiki/pmwiki.php/Literature/Zorro', '/pmwi...","['Zorro-like', 'Anpanman', 'Zorro', 'Bubblegum..."


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29903 entries, 0 to 29902
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   trope_name                  29903 non-null  object
 1   trope_link                  29903 non-null  object
 2   trope_description           29903 non-null  object
 3   related_link_names          29903 non-null  object
 4   related_links               29903 non-null  object
 5   example_descriptions        29903 non-null  object
 6   example_related_links       29903 non-null  object
 7   example_related_link_names  29903 non-null  object
dtypes: object(8)
memory usage: 1.8+ MB


In [17]:
df.replace('[]', np.nan, inplace=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29903 entries, 0 to 29902
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   trope_name                  29903 non-null  object
 1   trope_link                  29903 non-null  object
 2   trope_description           29903 non-null  object
 3   related_link_names          29822 non-null  object
 4   related_links               29822 non-null  object
 5   example_descriptions        26677 non-null  object
 6   example_related_links       26675 non-null  object
 7   example_related_link_names  26675 non-null  object
dtypes: object(8)
memory usage: 1.8+ MB


In [20]:
null_rows = df[df['related_link_names'].isnull()]

In [21]:
null_rows

,trope_name,trope_link,trope_description,related_link_names,related_links,example_descriptions,example_related_links,example_related_link_names
43,About The Author,http://tvtropes.org/pmwiki/pmwiki.php/Main/Abo...,['http://tvtropes.org/pmwiki/pmwiki.php/Main/A...,NaN,NaN,NaN,NaN,NaN
107,A Cappella,http://tvtropes.org/pmwiki/pmwiki.php/Main/ACa...,['http://tvtropes.org/pmwiki/pmwiki.php/Main/A...,NaN,NaN,"['In the anime version of The iDOLM@STER , Pre...","['/pmwiki/pmwiki.php/VideoGame/TheIdolMaster',...","['The iDOLM@STER', 'Archie Comics', 'Ultra Fas..."
1350,Appeal To Authority,http://tvtropes.org/pmwiki/pmwiki.php/Main/App...,['http://tvtropes.org/pmwiki/pmwiki.php/Main/A...,NaN,NaN,NaN,NaN,NaN
1358,Appeal To Popularity,http://tvtropes.org/pmwiki/pmwiki.php/Main/App...,['http://tvtropes.org/pmwiki/pmwiki.php/Main/A...,NaN,NaN,NaN,NaN,NaN
1559,Artistic License Linguistics,http://tvtropes.org/pmwiki/pmwiki.php/Main/Art...,['http://tvtropes.org/pmwiki/pmwiki.php/Main/A...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
28393,Video Game Weapon Stats,http://tvtropes.org/pmwiki/pmwiki.php/Main/Vid...,['http://tvtropes.org/pmwiki/pmwiki.php/Main/V...,NaN,NaN,NaN,NaN,NaN
28515,Violin Scam,http://tvtropes.org/pmwiki/pmwiki.php/Main/Vio...,['http://tvtropes.org/pmwiki/pmwiki.php/Main/V...,NaN,NaN,"[""Django Unchained : Dr Schultz pulls a loose ...","['/pmwiki/pmwiki.php/Film/DjangoUnchained', '/...","['Django Unchained', 'Blood Sport', 'Hold Your..."
28892,Webisode,http://tvtropes.org/pmwiki/pmwiki.php/Main/Web...,['http://tvtropes.org/pmwiki/pmwiki.php/Main/W...,NaN,NaN,['A few of General Mills \' different brands w...,['/pmwiki/pmwiki.php/UsefulNotes/GeneralMills'...,"['General Mills', 'Trix Toons', 'Azumanga Daio..."
29142,Where When Who,http://tvtropes.org/pmwiki/pmwiki.php/Main/Whe...,['http://tvtropes.org/pmwiki/pmwiki.php/Main/W...,NaN,NaN,NaN,NaN,NaN


In [24]:
filtered_row = null_rows[null_rows['trope_name'] == 'A Cappella']
trope_description_value = filtered_row['trope_description'].values
print(filtered_row)
# Display the value
print(trope_description_value)

     trope_name                                         trope_link  \
107  A Cappella  http://tvtropes.org/pmwiki/pmwiki.php/Main/ACa...   

                                     trope_description related_link_names  \
107  ['http://tvtropes.org/pmwiki/pmwiki.php/Main/A...                NaN   

    related_links                               example_descriptions  \
107           NaN  ['In the anime version of The iDOLM@STER , Pre...   

                                 example_related_links  \
107  ['/pmwiki/pmwiki.php/VideoGame/TheIdolMaster',...   

                            example_related_link_names  
107  ['The iDOLM@STER', 'Archie Comics', 'Ultra Fas...  
['[\'http://tvtropes.org/pmwiki/pmwiki.php/Main/ACappella\', \'Go To\', "Image selected per Image Pickin\' thread: https://tvtropes.org/pmwiki/posts.php?discussion=1584998622073193500Please do not replace or remove without starting a new thread.&#010;&#010;", \'A cappella is, like many musical terms, a phrase with multiple def